In [122]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
from selenium.webdriver.common.keys import Keys

browser = webdriver.Chrome()
browser.get('https://www.vip.com')
search_box = browser.find_element_by_css_selector('#J-search > div.c-search-form > input')
search_box.send_keys('手机\n')
sleep(3)
fout = open('唯品会手机商品数据爬虫.txt','w',encoding='utf-8')
k = 1

total_number = int(sp.select_one('span.total-item-nums').string[1:-1])
print(total_number)

for num in range(total_number):
    head = browser.find_element_by_tag_name('html')
    for _ in range(3):
        head.send_keys(Keys.END)
        sleep(1)
    html = browser.page_source
    sp = BeautifulSoup(html,'lxml')
    tag = sp.select_one('#J_searchCatList')
    goods = tag.select('div.c-goods-item')
    for item in goods:
        item_url = 'https:' + item.a.get('href')
        sale_price = item.select_one('div.c-goods-item__sale-price').get_text()
        t = item.select_one('div.c-goods-item__market-price')
        market_price = '' if t == None else t.get_text()
        t = item.select_one('div.c-goods-item__discount') 
        discount = '' if t == None else t.get_text()
        name = item.select_one('div.c-goods-item__name').get_text()  
        print(k,name,sale_price,market_price,
              discount,item_url,sep='#',file=fout)
        k += 1
    if num == total_number-1:
        break
    next_button = sp.select_one('#J_nextPage_link')
    next_url = 'https:' + next_button.get('href')
    browser.get(next_url)
    sleep(3)
fout.close()

df = pd.read_csv('唯品会手机商品数据爬虫.txt',sep='#',
                 names=['序号','名称','卖价',
                        '定价','折扣','商品网址'])
df.to_excel('唯品会手机商品数据爬虫.xlsx',index=False)

28
